<font size="3">
<center><h1>Introduction </h1></center>

> <b>Competition Setting: </b>Predicting Future Sales for a Russian software retail company 1C; Given a timeseries starting 2013-2015 Oct for sales of 1C on various items in it's multiple shop outlets, the ask is to predict monthly sales for the various shop-item pairs in Nov 2015.<br>
> <b>Why is this interesting for 1C: </b>This is a crucial question to start any data analytics process with; One of the biggest benefits data science delivers to retail companies is optimizing their inventory and procurement.
Thanks to predictive tools, businesses can use historical data and trend analysis to determine which products they should order-> they can optimize inventory management to emphasize products customers need effectively.
><center><h1>Dataset </h1></center>

Let me begin by giving a flavor of the beast we are dealing with here, the very voluminous but relatively easily comprehensible datasets we are to work with-;

> <b>sales_train.csv - </b>the training set. Daily historical data from January 2013 to October 2015.<br>
> <b>test.csv - </b>the test set. We need to forecast the sales for these shops and products for November 2015.<br>
> <b>items.csv - </b>supplemental information about the items/products (item names)<br>
> <b>item_categories.csv - </b>supplemental information about the items categories (item category names)<br>
> <b>shops.csv - </b>supplemental information about the shops (shop names)<br>

<b>Key Data Challenges - </b>Aside from the common issues of outliers and missing data (as we will see, the data quality provided here is actually very good, so we don't have to worry about these issues very much), there are 3 unique (or maybe not actually) data challenges-;

> <b>Russian Data - </b>Nearly all of the data is in Russian text, however we as data explorer junkies are not to be deterred by this! Here I will show how we can easily translate all of the text to English and the amazing insights this will lead us to, just as all the breathtaking stories only data hows to tell.<br>
> <b>Large Data - </b>The sales_train dataframe comprises of 3 million rows, which obfuscates signal detection to some extent but also obviously provides us with a tremendous opportunity to get more data insights and to pull our big guns out and brag with our more complex data forecasting processes. However for achieving this, we must endeavour to trim the noise from the data as far as possible.<br>
> <b>Mix of shops and items Data (with some Test data items unseen in Train) - </b>We are presented with transactions of various items and various shops in our train data set. Our model must be versatile enough so that it can build a wholesome model that can find ways to consistently predict despite these variations. Moreover it must be able to predict sales for items for which we have no data in our train set too!
><center><h1>Motivation </h1></center>

> I very recently embarked on this exciting data science expedition and ever since I have wondered why did I not delve into this intricate world before. This kernel (as part of a 2 kernels series) is intended to showcase (and more importantly to brag - I am <b>currently placed 12/11789 place (top 0.1%) with 0.821 RMSE score</b>) how much I enjoyed working on this data challenge and the unbelievably massive learning I had (will talk more about my learnings in the 2nd kernel); but it is also my hope that I can reveal some really exciting and breathtaking insights about this seemingly run-of-the-mill sales data, so that the reader is engaged and exhilarated enough to want to venture on working on this data themselves too and join in on the data adventures here!<br><br>
> As is custom, I began working with EDA, in order to invoke ideas for feature engineering, which as we know, is one of the most (if not THE most) significant processes for achieving good forecasting accuracy - here I have presented a snapshot of some of the most beautiful data visualizations which led me to some very unique feature engineering ideas and the final forecasting score that I achieved. Wondrously, this dataset offered the opportunity to leverage a huge variety of feature engineering tools- I have displayed here the many features I developed- <b>geolocation, text vectorization,log transformations,discretization of continuos features and target mean encodings of categorical features</b>.<br>
><center><h1>Acknowledgements </h1></center>

>I would like to thank the author of the following kernel from whom I learned so much, you must visit it as well-;
> - https://www.kaggle.com/kyakovlev/1st-place-solution-part-1-hands-on-data

>Without further ado, let us dive into the data analysis as promised!

In [ ]:
!pip install translate
!pip install folium
!pip install geopy
!pip install yellowbrick
!pip install google_trans_new

<font size="3">
<center><h1>Importing Libraries..

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

from google_trans_new import google_translator

from geopy.geocoders import Nominatim
import folium

from sklearn.feature_extraction.text import TfidfVectorizer
from yellowbrick.text import FreqDistVisualizer
from yellowbrick.text import TSNEVisualizer
from matplotlib.colors import ListedColormap
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction import text 
from operator import itemgetter

In [ ]:
sales=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
index_cols = ['shop_id', 'item_id', 'date_block_num']

sales.head()

<font size="3">

>This is the primary dataframe we will work with, as aforementioned the item,item category and shop names can all be derived from the other supplemental dataframes, and we will get to that soon enough...Let us first examine the distributions of the two numeric features given to us, the item price and item count day (target).

In [ ]:
sales['item_cnt_day'].describe()

<font size="3">

>First observation to make here is that there are negative entries for our target variable here as well, and this is because this represents the items returned. It is a viable option to try to treat these by suitably getting rid of them, however I am actually going to opt to leave them as is since <b>are looking to forecast monthly sales, not daily sales, so when we aggregate these daily sales to monthly (to have a uniform time frame as the test dataset),</b>these should automatically be squared off against the original sales. We will do a check at that point, to make sure this happens as we expect it to, of course.
>
>We also see the mean>median, so we know this distribution is skewed to the right, perhaps a log transformation would be nice to run here. Let us also visualize the distribution with a violin (yes you read that right).

In [ ]:
#histogram
sns.violinplot(sales['item_cnt_day'],split=True)
plt.title("Item Sales Distribution")

In [ ]:
sales['item_cnt_day'].value_counts(bins=3)

<font size="3">

>Most values are concentrated until ~400. 2 of the values are really extreme->we can be reasonably confident that they are outliers and would not help the forecasts in any way. I mean sales of >700 in a single transaction of an item by a software retail company seems super unlikely. Let's get rid of them.

In [ ]:
sales = sales[sales['item_cnt_day'] <= 700]
sales['item_cnt_day'].value_counts(bins=2)

In [ ]:
sns.distplot(sales['item_cnt_day'],kde=True,bins=10)
plt.title("Item Sales Distribution")

<font size="3">

>As we can see from the value counts and plot above, the distribution looks a bit more reasonable after removing the outliers.

In [ ]:
print("Skewness: %f" % sales['item_cnt_day'].skew())
print("Kurtosis: %f" % sales['item_cnt_day'].kurt())

<font size="3">

>The skewness and kurtosis are clearly too high, using a log transformation here is a good idea. Let us see if this is the case with our other numeric feature too.

In [ ]:
sns.violinplot(sales['item_price'])
plt.title("Item Price Distribution")

In [ ]:
sales['item_price'].value_counts(bins=6)

<font size="3">

>As we can see above, the same problems seem to plague item price as well. But unlike item sales, here there is no reason for us to have negative item prices (I say this but I remember the last time I was shifting apartments, I <b>paid</b> a fortune so that somebody would buy and take away that worn-out sofa from me! True story). Let us get rid of the negative and extremely high values. 

In [ ]:
sales = sales[(sales['item_price'] > 0) & (sales['item_price'] < 51000)]
sns.distplot(sales['item_price'],kde=True,bins=10)
plt.title("Item Price Distribution")

In [ ]:
print("Skewness: %f" % sales['item_price'].skew())
print("Kurtosis: %f" % sales['item_price'].kurt())

<font size="3">

>There, the distribution plot is much more reasonable now, however once again we see a significant skew. Again using a log transformation is a good idea when we prepare the data for training our models.All these fancy data distributions, why can't they just be <i>normal</i>?
>
>Let us now look at how these 2 numeric features have developed over time. But since I am going to be aggregating the data in a monthly time frame (this is the one we are most interested in); instead of plotting item price, I am going to use a factor of item price and sales count, 'Revenue'.

In [ ]:
sales['Revenue']=sales['item_cnt_day']*sales['item_price']
fig, axes = plt.subplots(2, 1, figsize=(25,15), sharex=True)
colors={'item_cnt_day':'green','Revenue':'red'}
for name, ax in zip(['item_cnt_day','Revenue'], axes):
    ax.plot(sales.set_index('date_block_num')[name], marker='.', linestyle='-', linewidth = 0.5, label=name, color=colors[name])
    #sns.boxplot(data = sales, x='date_block_num', y=name, ax=ax)
    ax.set_ylabel("",fontsize=20)
    ax.set_xlabel("Month",fontsize=20)
    ax.set_title(name,fontsize=20)
    ax.grid()
    if ax != axes[-1]:
        ax.set_xlabel('')

<font size="3">

>As for the trends that we can observe for the 2 timeseries plots, we see a general increase in item sales and Revenue over time. The advantage of having the luxury of being able to create both the plots sharing the X-axis, thanks again to Matplotlib, is that we can see a very interesting trend of negative correlation between Revenue and item sales, obviously item price and item sales seem to exhibit some comovement; which is intuitive as well- bigger bulk of products are generally sold at lower prices and vice versa.
>
>Also we can clearly observe a tinge of seasonality, indicating an need for taking this into account while training our models. I will do so by creating a very simple 'month' feature, so that each transaction is linked to the month of that transaction.

In [ ]:
sales['month']=sales['date_block_num']%12+1
sales['month'].unique()

<font size="3">

>Finally let us do a spot check for any missing data in our dataframe.

In [ ]:
print("Missing Values in Sales Data:")
sales.isna().sum()

<font size="3">

>Like I said earlier, our data is actually extremely clean,now let's move towards analyzing some of the supplemental datasets. That's where I promise our analysis will get so much more creative and amazing!

In [ ]:
shops=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
shops['shop_name'].unique()

<font size="3">

>From the shops dataframe, we see that all names are in Russian, so unless we have all native speakers here, we must translate it all to extract some interesting features from the shop names.

In [ ]:
shops=pd.read_csv("../input/1c-translated-shops/Translated_shops.csv")
shops=shops[shops.columns[1:]]
sales=sales.join(shops.set_index('shop_id'),on='shop_id',how='left')
""" 
##Below code is how I completed the translations

translator = google_translator()
import time

shop_translations = {}
unique_elements = shops['shop_name'].unique()
for element in unique_elements:
    # add translation to the dictionary    
    shop_translations[element] = translator.translate(element)

shops['Trans_shops']=shops['shop_name'].map(shop_translations)
sales['Trans_shops']=sales['shop_name'].map(shop_translations)
"""
sales['Trans_shops'].unique()

<font size="3">

>There, isn't that so much better, finally we see something that we can use . Already we can make a significant observation here; the 'Yakutsk' shop, seems to be really similar in name. After doing some comprehensive checking of all the shop names, I could find 4 shops that are very identically named-;

In [ ]:
print(sales.loc[sales['shop_id']==0,'Trans_shops'].iloc[0],sales.loc[sales['shop_id']==57,'Trans_shops'].iloc[0])
print(sales.loc[sales['shop_id']==1,'Trans_shops'].iloc[0],sales.loc[sales['shop_id']==58,'Trans_shops'].iloc[0])
print(sales.loc[sales['shop_id']==11,'Trans_shops'].iloc[0],sales.loc[sales['shop_id']==10,'Trans_shops'].iloc[0])
print(sales.loc[sales['shop_id']==40,'Trans_shops'].iloc[0],sales.loc[sales['shop_id']==39,'Trans_shops'].iloc[0])

In [ ]:
print(sales.loc[sales['shop_id']==0,'shop_name'].iloc[0],sales.loc[sales['shop_id']==57,'shop_name'].iloc[0])
print(sales.loc[sales['shop_id']==1,'shop_name'].iloc[0],sales.loc[sales['shop_id']==58,'shop_name'].iloc[0])
print(sales.loc[sales['shop_id']==11,'shop_name'].iloc[0],sales.loc[sales['shop_id']==10,'shop_name'].iloc[0])
print(sales.loc[sales['shop_id']==40,'shop_name'].iloc[0],sales.loc[sales['shop_id']==39,'shop_name'].iloc[0])

<font size="3">

>As we can see from both the English and original Russian shop names (yayyy I can read Russian now too!), this almost certainly is an error, the names are near identical and is probably a data entry typo. But just to be double sure let us examine the time periods these shops are present in to check if they are interspersed.

In [ ]:
print(str(sales.loc[sales['shop_id']==0,'date_block_num'].unique()),str(sales.loc[sales['shop_id']==57,'date_block_num'].unique()))
print(str(sales.loc[sales['shop_id']==1,'date_block_num'].unique()),str(sales.loc[sales['shop_id']==58,'date_block_num'].unique()))
print(str(sales.loc[sales['shop_id']==11,'date_block_num'].unique()),str(sales.loc[sales['shop_id']==10,'date_block_num'].unique()))
print(str(sales.loc[sales['shop_id']==40,'date_block_num'].unique()),str(sales.loc[sales['shop_id']==39,'date_block_num'].unique()))

<font size="3">

>Just as I suspected, the time periods these shops are present in are mutually exclusive (shop nos. 40 and 39 do have some common periods but even for them in the last 10 months, there is only data for shop 39), further strengthening my suspicion. Damn, I would make a good detective, wouldn't you agree? But then again all data scientists like you would too.
>
>Let us set these shops to be the same, so that their data can be accurately aggregated and the shop sales are not misrepresented. I am going to set these shops to be equal to the ones which are in the Test Set, for ease in training and then forecasting.

In [ ]:
sales.loc[sales['shop_id']==0,'shop_name']=sales.loc[sales['shop_id']==57,'shop_name'].iloc[0]
sales.loc[sales['shop_name']==sales.loc[sales['shop_id']==57,'shop_name'].iloc[0],'shop_id']=57
sales.loc[sales['shop_id']==1,'shop_name']=sales.loc[sales['shop_id']==58,'shop_name'].iloc[0]
sales.loc[sales['shop_name']==sales.loc[sales['shop_id']==58,'shop_name'].iloc[0],'shop_id']=58
sales.loc[sales['shop_id']==11,'shop_name']=sales.loc[sales['shop_id']==10,'shop_name'].iloc[0]
sales.loc[sales['shop_name']==sales.loc[sales['shop_id']==10,'shop_name'].iloc[0],'shop_id']=10
sales.loc[sales['shop_id']==40,'shop_name']=sales.loc[sales['shop_id']==39,'shop_name'].iloc[0]
sales.loc[sales['shop_name']==sales.loc[sales['shop_id']==39,'shop_name'].iloc[0],'shop_id']=39

print("New unique shops count: "+str(sales['shop_id'].nunique()))

<font size="3">

>With that completed, our shops count has dropped and we can be more confident about the data in the context of shops. Now we can suitably scruitinize the shop wise data to make some inferences on the same, let's begin by checking which shops have the highest amount of transactions, and how skewed this distribution is, but this let's add some color to our plots, shall we?

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(30, 15))
sns.countplot('shop_id', data= sales,
                 order=sales['shop_id'].value_counts().index[:20],
                 alpha=0.7)
#ax[i//3][i%3].set_ylim([0, 250000])
ax.set_xlabel("shop_id",fontsize=20)
ax.set_ylabel("count",fontsize=20)
ax.grid()
ax.set_title('High Frequency shop_ids', fontsize=20)

<font size="3">

>Obviously not all shops have the same number of transactions, with some shops winning the bulk of the transactions. However the important question we must ask ourselves is does the highest number of transactions translate to highest number of item sales (our target variable, after all)? 

In [ ]:
fig = plt.figure(figsize=(40, 20))

plt.title("Do the shops with most frequent sales have the highest sales?",fontsize=30)

ax1 = fig.add_subplot(2,1,1)
sns.countplot(data = sales, x = 'shop_id', ax = ax1)
ax1.set_xlabel("shop_id",fontsize=20)
ax1.set_ylabel("count",fontsize=20)
ax1.grid()

ax2 = fig.add_subplot(2,1,2)
sns.boxplot(data = sales, x='shop_id', y='item_cnt_day' , ax = ax2)
ax2.set_xlabel("shop_id",fontsize=20)
ax2.set_ylabel("item_cnt_day",fontsize=20)
ax2.grid()

<font size="3">

>Suprise, suprise! The number of transactions seems to be inversely related (loosely speaking) to the number of total item sales by the shops. This is a useful discovery, because going forward (in my 2nd kernel), I use the number of transactions per month for each shop as a feature, and it definitely has significance.
>
>An important challenge of our dataset as I mentioned before is that we are presented with transactions for a large amount of shops, are all of these shops present in our test set? Let us investigate.

In [ ]:
test=pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
print("We have a total of "+str(test['shop_id'].nunique())+" shops in our test set.")

<font size="3">

>So only 42 of the 56 shops of our train set are present in the test set. This begs a very pivotal question, especially given the already large amount of train data, whether should we really retain the data for those 14 shops absent in the test set? Would they really add value to our forecasting process for the test set? Even if they do add some value, do they justify the vast amount of baggage (large data) they carry?
>
>These are some very tricky questions to approach, and as usual the answer lies within our data itself. Let us try to primarily examine for 2 traits within the shops data-;
>
> - How much value do these extra 14 shops add to the forecasting of the rest of the 42 shops test data. We need to find a way to quantify this, or even better, visualize it.
> - How much noise do these 14 shops have in their data?

In [ ]:
shopwise_sales=pd.DataFrame(sales.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].sum()).reset_index()

shopwise_sales['month_item']=shopwise_sales['date_block_num'].astype('str')+'_'+shopwise_sales['item_id'].astype('str')
del shopwise_sales['date_block_num'],shopwise_sales['item_id']

shopwise_sales=shopwise_sales.set_index('month_item').pivot(columns='shop_id',values='item_cnt_day').fillna(0.0)
shopwise_sales.head()

<font size="3">

>Finally now that our pivoted shopwise dataset is ready in the format I like, I have the license to get the first big weapon out of my Seaborn arsenal! 

In [ ]:
corrmat = shopwise_sales.corr()
f, ax = plt.subplots(figsize=(16, 12))
sns.heatmap(corrmat, vmax=.8, square=True);
plt.title("Shop Sales Heatmap",fontsize=20)

<font size="3">

>To get a good look in, we are going to have to squint a bit, but it's highly rewarding ~ in this one plot, we can examine the correlations between all given shops with each other in terms of their item sales!
>
>To start off, we see majority of the shops are light skin-peach-ish colours, indeed indicating a good correlation among shop sales.
>
>There are a few red/purple patches as well, but the worst of them all are the black ones. Let's turn our attention to these black shops, which indeed do seem likely to cast only darkness to our predictive models. If we squint hard enough, we can see there are 4 shops with those black lines - shops 9,20,36 and 55. Unfortunately shops 36 and 55 are present in our test set, so we are going to have to stick with those, however shops 9 and 20 seem to only bring noise in our train data set,let's see their shop names. 

In [ ]:
print(sales.loc[(sales['shop_id']==9),'Trans_shops'].iloc[0])
print(sales.loc[(sales['shop_id']==20),'Trans_shops'].iloc[0])

<font size="3">

>These seem to be some special "Sale" and "Exit" shops, that could explain why their transactions data is so unusual, and uncorrelated with the rest of the train data, let's get rid of these shops transactions data already. See I always say more data is not always better!... Ok thats a lie, I love data and I believe I can never have enough of it.
>
>Note: To be doubly sure, I did cross validation of my models both with and without these 2 shops as well, and no prizes for guessing, my models without these insolent shop data points fared way better. 

In [ ]:
sales=sales[(sales['shop_id']!=9)&(sales['shop_id']!=20)]

<font size="3">

>Now let us try to see if we can find some meaningful features from the shop names data. Something that immediately catches our eye, is the first word of all the shop names...most of them seem to belong to a city, we can definitely tell that about Moscow. Let us once again get our data in a more friendly format..

In [ ]:
shopwise_sales=pd.DataFrame(sales.groupby(['date_block_num','Trans_shops','item_id'])['item_cnt_day'].sum()).reset_index()
shopwise_sales['month_item']=shopwise_sales['date_block_num'].astype('str')+'_'+shopwise_sales['item_id'].astype('str')
del shopwise_sales['date_block_num'],shopwise_sales['item_id']
shopwise_sales=shopwise_sales.set_index('month_item').pivot(columns='Trans_shops',values='item_cnt_day').fillna(0.0)
shopwise_sales['shop']=shopwise_sales.index.str.split('_')
shopwise_sales['shop']=shopwise_sales['shop'].str[1]
shopwise_sales.head()

<font size="3">

>From the above pivoted snapshot, we can easily see so many first names to be repeated among shop names, giving voice to my educated guess.
>
>I don't know about you readers, but I have little to no idea about the Russian geography, so once again to validate my guess, I am going to again dazzle and overwhelm our senses with a huge amount of data in a small space, so we better have our wits about to draw appropriate inferences!

In [ ]:
sns.set(font_scale = 0.75)
cols = shopwise_sales.columns[:3].tolist()+shopwise_sales.columns[11:13].tolist()
sns.pairplot(shopwise_sales[cols], size = 2.5, plot_kws= {"s":40,"alpha":1.0,'lw':0.5,'edgecolor':'k'})

plt.show();

<font size="3">

>I have cherry picked 5 shops here, 4 of them belonging to the same cities (in 2 pairs) and 1 of them an online shop. Above I have the scatter plots of all these shops' sales data against one another, as we can see the plots confirm that there is indeed use for having a 'city' feature.
>
>As we see the Yakutsk city plots (first 2) and Krasnoyarsk city plots (last 2) are the only ones showing correlation in their scatter plots. The Yakutsk shops are uncorrelated with Krasnoyarsk shops and all the shops are uncorrelated with the online shop sales (middle row of plots) where we those L-shaped plots. The L-shaped plots are once again a reminder of how all the data is so skewed towards item sales of very small amounts. Thus we see a lot of promise for the 'city' feature which can so easily be formulated just as the first word of the shop names.

In [ ]:
sales['city']=sales['Trans_shops'].str.split(' ').str[0]
sales['city'].unique()

<font size="3">

>With the 'city' feature in hand, once again if we scrutinize all the city names, we find that some cities are very similar with minute changes. This is because obviously we cannot rely on perfect translations from the Python Google library (that's right Google, you Suck!! Wait isn't Kaggle owned by Google now? Oh no..)
>
>For example, Voronez seems awfully similar to Voronezh and SPB to SPb, even though I have no clue where on Earth these cities are (I will remedy that as well in a bit). Also 'Online' to '1c=online', let's right these wrongs...

In [ ]:
sales.loc[sales['city']=='Voronež','city']='Voronezh'
sales.loc[sales['city']=='SPb','city']='SPB'
sales['city']=np.where(sales['city'].str.contains('online'),'Internet',sales['city'])
sales['city']=np.where(sales['city'].str.contains('Online'),'Internet',sales['city'])
sales['city']=np.where(sales['city']=='Rostnone','Rostovnadon',sales['city'])

sales.loc[sales['city'].str.contains('Internet'),'shop_id'].unique()

<font size="3">

>Hmmm 12 and 55, why do they sound so familiar? Yes that's right if you scroll up to our correlation heatmaps, you will see 55 is another of those rogue black shops spreading darkness to our dataset, and 12 is pretty purple too. But since they are unfortunately present in our test set too, we have to stick with them and perhaps find another way to model them. Knowing that both of them belong to the category of Internet shops should help.
>
>Anyhow let's see what we can do about getting to those locations of these cities we have generated. I am a visual person and I like it when I am able to perfectly visualize what I am forecasting here.

In [ ]:
geolocator = Nominatim(user_agent='myapplication')

# Get a basic world map.
shops_map = folium.Map(location=[60,40], zoom_start=3,tiles="Stamen Terrain")
for rus_city in sales['city'].unique():
    if rus_city=='Sergiev':
        rus_city= 'Sergiev Posad'
    location = geolocator.geocode(rus_city)
    if location!=None:
        folium.CircleMarker(location=[location.raw['lat'], location.raw['lon']], popup=rus_city,radius=2.5, 
                    color='yellow',
                    fill=True,
                    fill_color='blue',
                    fill_opacity=0.6).add_to(shops_map)
shops_map

<font size="3">

>That is a satisfying visualization to leave an imprint of all our shops in our mind, while we train our models. I think the shops in Austria ("Kolo") and France ("Ufa") are misrepresented here, so we can ignore them, but for the other cities, we can easily tell that even the cross-location distances, for eg. can be a great feature.
>
>That's a lot of food for thought from the shops data, let's now leap onto the items' data and direct our detective lens in that direction.

In [ ]:
item=pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
item_cat=pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
test=test.join(item.set_index('item_id'),on='item_id',how='left')
test=test.join(item_cat.set_index('item_category_id'),on='item_category_id',how='left')

sales=sales.join(item.set_index('item_id'),on='item_id',how='left')
sales=sales.join(item_cat.set_index('item_category_id'),on='item_category_id',how='left')

itemcat_sales=pd.DataFrame(sales.groupby(['date_block_num','shop_id','item_category_id'])['item_cnt_day'].sum()).reset_index()
itemcat_sales['month_shop']=itemcat_sales['date_block_num'].astype('str')+'_'+itemcat_sales['shop_id'].astype('str')
del itemcat_sales['date_block_num'],itemcat_sales['shop_id']
itemcat_sales=itemcat_sales.set_index('month_shop').pivot(columns='item_category_id',values='item_cnt_day').fillna(0.0)
itemcat_sales.head()

In [ ]:
print("Test set contains "+str(test['item_category_id'].nunique())+" item categories")

<font size=3>

>Once again there are 84 item categories and not all of them are present in our test set (which has only 62 of them). So let us revisit our favorite heatmap once again, this time looking at the item categories' sales data pitted against each other.

In [ ]:
corrmat = itemcat_sales.corr()
f, ax = plt.subplots(figsize=(16, 16))
sns.heatmap(corrmat, vmax=.8, square=True);
plt.title("Item Categories Sales Heatmap",fontsize=20)

<font size=3>

>Oh my those purple corridors, I think even a truck could pass through them! The item categories' sales data seems much more uncorrelated in general with each other as opposed to that of shops, which intuitively makes a lot of sense. I mean how much can the same product be different in price and amount of sales among different shops really? Maybe some really entreprenuerial shop salesmen can do better than others, but for the same product, it cannot be that much different.
>
>Whereas if we talk about different item categories in the same shop, they could be as different as an 'Apple Iphone' and a Ps4 DvD for Witcher 3. One of these rightfully charges a small fortune whereas the other forces it's customers to sell a kidney (and a lung for it's accessories) to pay out a huge fortune (no I'm not going to specify which is which). And thus I really don't see a strong reason to hold on to the item categories' data for which there is no mention of in the test set. And yes once again I did cross validate my models with and without these 22 item categories, finding not a major difference in forecasting accuracy but definitely a boost in forecasting time.

In [ ]:
sales=sales.loc[sales['item_category_id'].isin(test['item_category_id'])]

<font size=3>

>Now let's translate all our item category names too, so we can make some sense out of those.

In [ ]:
cats=pd.read_csv("../input/1c-translated-item-categories/Translated_item_categories.csv")
cats=cats[cats.columns[2:]]
sales=sales.join(cats.set_index('item_category_id'),on='item_category_id',how='left')
""" 
##Below code is how I completed the translations

translator = google_translator()

cat_translations = {}
unique_elements = item_cat['item_category_name'].unique()
for element in unique_elements:
    # add translation to the dictionary
    cat_translations[element] = translator.translate(element)
    
item_cat['Trans_cat']=item_cat['item_category_name'].map(cat_translations)
sales['Trans_cats']=sales['item_category_name'].map(cat_translations)

"""

sales['Trans_cat'].unique()

<font size=3>

>I see a lot of similar categories, for eg the various 'Games' categories and obviously they are not the same but they can be expected to exhibit similar behaviour in terms of item sales and Revenue. Thus providing the potential to create a new feature which would perhaps be a broader category clubbing these similar categories together. Let's hold that thought in our minds for a little bit, I will get to it in just a minute. But first let's also translate the item names so it can easily be worked on. Note that I am not going to run the item names translations here because it took me ages to run that on my Personal PC. I will just load it here.

In [ ]:
item_translates=pd.read_csv("../input/1c-translated-items/Completed_item_translations.csv")
item_translates.columns=['item_name','Trans_item']
item_translates=dict(item_translates.values)
sales['Trans_item']=sales['item_name'].map(item_translates)

item_cat_best=sales.groupby(['Trans_cat','Trans_item'])['item_cnt_day'].sum().sort_values(ascending=False).reset_index()
item_cat_best[:10]

<font size=3>

>Armed with the translations of both categories and items now, let's get a bit inventive with how we visualize the top categories and items, and move away from the Seaborn sensation for a little bit.

In [ ]:
import plotly.express as px
fig =px.sunburst(
    item_cat_best[1:11],path=['Trans_cat','Trans_item'],
    values='item_cnt_day',title='Top 10 Item Category/Item name Pairs Sunburst'
)
fig.show()

<font size=3>

>Wow another unbelieveable method of summarising such a great deal of information in just a circle, I was truly infatuated when I came across this idea of visualizing the various categories as several rings of the circle, starting with the top 10 broader 'item categories' feature here, and on the outer half presenting the top items within each of these categories, and if we hover over any of them, we can even see the exact item sales values for each grouping.
>
>This Sunburst once again reiterates my earlier point of having similar categories, as we can see from the 3 'Games' categories in the various platforms: PC,PS3 and Xbox 360. All 3 of these 'Games' categories have 'Grand Theft Auto' as their top grossing product and these games can be expected to exhibit similar characteristics, so let's create a grouping for these similar categories and similar items.

In [ ]:
sales['Broad_cat']=sales['Trans_cat'].str.split('-').str[0]
sales.loc[sales['Broad_cat']=='PC games ','Broad_cat']='PC Games '
sales['Broad_cat'].unique()

<font size=3>

>There that's lesser than a quarter of the categories we started out with and it looks a lot more cleaner too. I have actually left categories like PC Games, Games, Android Games etc. separated here, but another very interesting paradigm to consider is to club all such categories together (the aforementioned categories could all be grouped under 'Games'), and make an additional feature of the platform the item belongs to (very relevant since this is a software retail company's data we are playing with).

In [ ]:
sales['Broad_item']=sales['Trans_item'].str.split(' ').str[0:2].str.join(',').str.replace(',',' ')
sales['Broad_item'].head()

In [ ]:
fig = plt.subplots(figsize=(16, 8))

plt.subplot(121)
custom_colors = sns.color_palette("Spectral")
broadcat_sales=sales.groupby('Broad_cat')['item_cnt_day'].sum().sort_values(ascending=False).reset_index()[:10]
plt.pie(broadcat_sales['item_cnt_day'],labels=broadcat_sales['Broad_cat'],radius = 1.5,\
        explode=(0,0,0,0,0,0,0,0,0,0),autopct='%1.1f%%',colors=custom_colors,shadow=True)
centre_circle = plt.Circle((0,0),0.65,color='black', fc='white',linewidth=1.25)
plt.title("Top Broad cats donut chart",fontsize= 30,fontname="Times New Roman",fontweight="bold")
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.axis('equal')
fig.tight_layout()

plt.subplot(122)
import matplotlib.colors as colors
custom_colors = sns.color_palette("Paired")
broaditem_sales=sales.groupby('Broad_item')['item_cnt_day'].sum().sort_values(ascending=False).reset_index()[1:11]
plt.pie(broaditem_sales['item_cnt_day'],labels=broaditem_sales['Broad_item'],radius = 1.5,\
        explode=(0.215,0.4,0.15,0.3,0.25,0,0.03,0.1,0.01,0.07),autopct='%1.1f%%',colors=custom_colors,startangle=180)
plt.title("Top Broad items pie chart",fontsize= 30,fontname="Times New Roman",fontweight="bold")
plt.axis('equal')
fig.tight_layout()
plt.show()

<font size=3>

>For all you enthusiastic foodies, isn't it so awesome that we can fix our appetite with these data appetizers, served as donuts and pies, I think I am already drooling. Making Broad Item categories has helped us to easily see the most selling items. For instance the games are obviously the biggest sales for 1C, here we see the games just have the first 2 words of their name (making some of them weird but still uniquely identifiable - for all you gamers, "Need for" is obviously NFS (Need For Speed), "Call of " is obviously "Call of Duty"); a fact that would be harder to prove just by seeing the pie charts on individual items, since that would be spread across various platforms.
>
>Let's now move across to unsheath another of our big weapons from our data armoury and perform some text analysis.

In [ ]:
items_subset = item[['item_id', 'item_name','item_category_id']]
items_subset['Trans_item']=items_subset['item_name'].map(item_translates)

##This is the set of tf-idf features I go on to use for the models I train for my final predictions
feature_count = 25
tfidf = TfidfVectorizer(max_features=feature_count)
tfidf = TfidfVectorizer()
items_text_fts = pd.DataFrame(tfidf.fit_transform(items_subset['item_name']).toarray())

cols =items_text_fts.columns
for i in range(feature_count):
    feature_name = 'item_name_tfidf_' + str(i)
    items_subset[feature_name] = items_text_fts[cols[i]]

In [ ]:
vectorizer = CountVectorizer(stop_words=text.ENGLISH_STOP_WORDS.union(stopwords.words('russian')))
docs = vectorizer.fit_transform(items_subset['item_name'])

word_list = vectorizer.get_feature_names()
count_list = docs.toarray().sum(axis=0)
vals=dict(zip(word_list, count_list))

N = 15
res = dict(sorted(vals.items(), key = itemgetter(1), reverse = True)[:N])

features = vectorizer.get_feature_names()

new_vals=[]
rus_vals=[]
for element in res.keys():
    rus_vals.append(element)
    new_vals.append(element)

for n, i in enumerate(features):
    for count in range(N):
        if i==rus_vals[count]:
            features[n]=rus_vals[count]   
            
visualizer = FreqDistVisualizer(features=features,n=N,color=custom_colors)
visualizer.fit(docs)
visualizer.poof()

<font size=3>

>Starting out with a simple CountVectorizer, I had to be sure to do the count on Russian words itself (translation always loses some value), remove the stop words of both the English and Russian dictionaries and then convert the words back to English for us; finally thanks to YellowBrick, we have it neatly plotted into a Frequency Distribution visualizer. Once again we see how useful a platform feature will be for our prediction here (will generate it in 2nd kernel), with most keywords playing around pc,ps3,xbox and even MP3!
>
>Let's now move across to generating features from the item names text (I found the item names to be the most useful text feature for deriving robust predictive features) using TfIdf Vectorizer from Sklearn. To visualize it, I am going to discretize item sales too.

In [ ]:
sample_sales=sales.loc[sales['item_cnt_day']<=1.0].iloc[:500]
sample_sales=sample_sales.append(sales.loc[(sales['item_cnt_day']>20.0)&(sales['item_cnt_day']<30.0)].iloc[:500])
sample_sales=sample_sales.append(sales.loc[sales['item_cnt_day']>45.0].iloc[:500])
sample_sales['binned_item_cnt']=pd.qcut(sample_sales['item_cnt_day'].rank(method='first'),q=3)

tsne = TSNEVisualizer()

tsne.fit(tfidf.fit_transform(sample_sales['item_name']),sample_sales['binned_item_cnt'],color=sns.color_palette("Spectral"),\
cmap = ListedColormap(('red', 'green')))
tsne.poof()

<font size=3>

>The binning of the item sales meant that I was able to plot the text features such that we can see whether these features were able to divvy up the binned vector into clusters. But since item sales were heavily right skewed, I had to individually pick a set of sample transactions to ensure there was sufficient dispersion among them while plotting TSNE graph. And we can see some blobs of the 3 colours being formed, providing evidence of the efficacy for using item text features.
>
>Finally let us move to a very interesting observation I made with the help of another timeseries plot which can be described as how 1C has grown it's items and shops base with time, which assisted me a great deal in doing some post-processing for improving my model forecasts.

In [ ]:
sales['num_uniq_items']=sales.groupby('date_block_num')['item_id'].transform(lambda x: x.nunique())
sales['num_uniq_shops']=sales.groupby('date_block_num')['shop_id'].transform(lambda x: x.nunique())

fig, axes = plt.subplots(2, 1, figsize=(16, 9), sharex=True)
colors={'num_uniq_items':'coral','num_uniq_shops':'purple'}

for name, ax in zip(['num_uniq_items','num_uniq_shops'], axes):
    ax.plot(sales.set_index('date_block_num')[name], marker='.', linestyle='-', linewidth = 0.5, label=name, color=colors[name])
    #sns.boxplot(data = sales, x='date_block_num', y=name, ax=ax)
    ax.set_ylabel("")
    ax.set_xlabel("Month")
    ax.set_title(name)
    if ax != axes[-1]:
        ax.set_xlabel('')

<font size=3>

>The above timeseries plots are great to demonstrate how 1C shops' sales have developed over time (given the data we have). We see a clear decrease in both the number of item offerings and operating shops for 1C; this raises a very important question, namely how many items are in test set that could be such that they are now outdated,i.e have not been sold by any of the shops since past few months (shout out to kyakovlev for introducing this idea as well)? Let us do some analysis to find out, once again in my favorite dataframe format, the pivoted tables.

In [ ]:
pivoted_item_sales = sales.pivot_table(index=['item_id'],values=['item_cnt_day'], 
                                        columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
pivoted_item_sales.columns = pivoted_item_sales.columns.droplevel().map(str)
pivoted_item_sales = pivoted_item_sales.reset_index(drop=True).rename_axis(None, axis=1)
pivoted_item_sales.columns.values[0] = 'item_id'

outdated_items = pivoted_item_sales[pivoted_item_sales.loc[:,'27':].sum(axis=1)==0]
print('Outdated items:', len(outdated_items))

print('Outdated items in test set:', len(test[test['item_id'].isin(outdated_items['item_id'])]))

<font size=3>
   
>The implication of the above finding is extremely significant! It means that there is a very strong chance those 6972 items in test set (across all shops) are likely to have zero sales for our test month (November), we can simply mark them as 0.0 for our predictions without even running any models! I cross validated the use of this logic for prior months, the most shocking fact I found is even the best models I could train did not make the zero predictions, which when I manually changed to zero, improved my cross validation error significantly. This just goes to show how important preprocessing,visualization,EDA and in general human intuition is, given that it can beat even the most complex models in existence (at least so far it can).
>
>And I think that is a very fitting note to end the 1st part of my kernels in, it's truly been extremely fun and joyous putting this together, and I have learned so much while attempting to articulately express my thought process with my work on this competition. I hope you readers have gained at least a little as well from the journey, if not then read it again! Jokes aside, this is my first public kernel, so <b>please do drop feedback on how I can improve it, and what you did/did not like</b>.